## 데이터 불러오기

In [0]:
import pandas as pd
import numpy as np

jdbc_url = "jdbc:sqlserver://1dt-team4-sqlserver.database.windows.net:1433;database=1dt-team4-sqldb"
connection_properties = {
    "user": "azureuser",
    "password": "team4123!@#",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


# 기준 데이터 (batch)
df_ref = spark.read.jdbc(url=jdbc_url, table="gold.gold_realscore_pred", properties=connection_properties).toPandas()

# 수치형 컬럼
numeric_cols = df_ref.select_dtypes(include=[np.number]).columns.tolist()
# 범주형 컬럼
categorical_cols = ['gender', 'grade']  # 전처리 시 감지했던 범주형

# 수치형 통계
ref_stats_numeric = df_ref[numeric_cols].agg(['mean', 'std']).T

# 범주형 통계
ref_stats_categorical = {}
for col in categorical_cols:
    ref_stats_categorical[col] = df_ref[col].value_counts(normalize=True).to_dict()

In [0]:
display(df_ref)

In [0]:
# Load as Spark DataFrame
df_recent = spark.read.jdbc(
    url=jdbc_url,
    table="gold.realtime_pred_realscore",
    properties=connection_properties
)

# Rename column using Spark DataFrame method
df_recent = df_recent.withColumnRenamed('pred_realScore_clean', 'realScore_clean')

# Convert to pandas DataFrame
df_recent = df_recent.toPandas()

# Drop columns if they exist
cols_to_drop = ['percent_rank', 'grade_percentile_calc']
for c in cols_to_drop:
    if c in df_recent.columns:
        df_recent = df_recent.drop(c, axis=1)

# 수치형 통계
new_stats_numeric = df_recent[numeric_cols].agg(['mean', 'std']).T

# 범주형 통계
new_stats_categorical = {}
for col in categorical_cols:
    new_stats_categorical[col] = df_recent[col].value_counts(normalize=True).to_dict()

In [0]:
import pandas as pd
import numpy as np

# 8학년, 9학년 샘플 데이터 추가 생성
more_rows = pd.DataFrame([
    {
        'learnerID': 'A080000333',
        'gender': 'F',
        'grade': 8,
        'testID': 'A080000003',
        'correct_cnt': 8,
        'items_attempted': 10,
        'difficultyLevel_mean': 0.31,
        'discriminationLevel_mean': 3.01,
        'guessLevel_mean': 0.18,
        'theta_clean': 0.52,
        'realScore_clean': 0.73
    },
    {
        'learnerID': 'A080000444',
        'gender': 'M',
        'grade': 8,
        'testID': 'A080000004',
        'correct_cnt': 5,
        'items_attempted': 9,
        'difficultyLevel_mean': -0.05,
        'discriminationLevel_mean': 2.77,
        'guessLevel_mean': 0.14,
        'theta_clean': -0.18,
        'realScore_clean': 0.49
    },
    {
        'learnerID': 'A090000555',
        'gender': 'F',
        'grade': 9,
        'testID': 'A090000005',
        'correct_cnt': 9,
        'items_attempted': 10,
        'difficultyLevel_mean': 0.55,
        'discriminationLevel_mean': 3.22,
        'guessLevel_mean': 0.21,
        'theta_clean': 0.78,
        'realScore_clean': 0.81
    },
    {
        'learnerID': 'A090000666',
        'gender': 'M',
        'grade': 9,
        'testID': 'A090000006',
        'correct_cnt': 4,
        'items_attempted': 8,
        'difficultyLevel_mean': -0.22,
        'discriminationLevel_mean': 2.63,
        'guessLevel_mean': 0.13,
        'theta_clean': -0.47,
        'realScore_clean': 0.36
    }
])

# df_recent에 추가
df_recent = pd.concat([df_recent, more_rows], ignore_index=True)

# 확인
print(df_recent.tail(10))


## 평균으로 감지

In [0]:
drift_flags = {}

# 수치형 간단 drift
for col in numeric_cols:
    ref_mean = ref_stats_numeric.loc[col, 'mean']
    new_mean = new_stats_numeric.loc[col, 'mean']
    drift_flags[col] = abs(new_mean - ref_mean) > 0.1 * abs(ref_mean)  # 10% 이상 변화면 drift

# 범주형 간단 drift
for col in categorical_cols:
    ref_dist = ref_stats_categorical[col]
    new_dist = new_stats_categorical[col]
    keys = set(ref_dist.keys()) | set(new_dist.keys())
    max_diff = max(abs(ref_dist.get(k,0)-new_dist.get(k,0)) for k in keys)
    drift_flags[col] = max_diff > 0.1  # 10% 이상 변화면 drift

print("Drift detected:", drift_flags)

In [0]:
print(df_ref[numeric_cols].dtypes)
print(df_recent[numeric_cols].dtypes)


## 수치형변수

ks-test: 두 연속형 분포가 같은지 검정
ks검정: 기존 학습 데이터 분포 vs 최근 데이터 분포 p-val < 0.05시 드리프트 존재

In [0]:
from scipy.stats import ks_2samp

drift_flags_numeric = {}
p_values_numeric = {}

for col in numeric_cols:
    stat, p_value = ks_2samp(df_ref[col], df_recent[col])
    drift_flags_numeric[col] = p_value < 0.05  # 유의수준 0.05
    p_values_numeric[col] = p_value

## 범주형변수

Chi-square test: 두 범주형 분포가 같은지 검정(범주별 비율 변화)

In [0]:
from scipy.stats import chi2_contingency

drift_flags_categorical = {}
p_values_categorical = {}

for col in categorical_cols:
    ref_counts = df_ref[col].value_counts().reindex(df_ref[col].unique(), fill_value=0)
    new_counts = df_recent[col].value_counts().reindex(df_ref[col].unique(), fill_value=0)
    contingency = pd.DataFrame({'ref': ref_counts, 'recent': new_counts})
    stat, p_value, _, _ = chi2_contingency(contingency + 1e-6)  # 0 방지
    drift_flags_categorical[col] = p_value < 0.05
    p_values_categorical[col] = p_value


## 시각화

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

# 제외할 컬럼
exclude_cols = ['grade', 'discriminationLevel_mean']

# numeric_cols에서 제외
numeric_cols_plot = [col for col in numeric_cols if col not in exclude_cols]

# 그리드 크기
n_cols = 3
n_rows = (len(numeric_cols_plot) + n_cols - 1) // n_cols  # 필요한 행 계산

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
axes = axes.flatten()  # 1D로 변환

for i, col in enumerate(numeric_cols_plot):
    ax = axes[i]
    sns.kdeplot(df_ref[col], label='Reference', fill=True, ax=ax)
    sns.kdeplot(df_recent[col], label='Recent', fill=True, ax=ax)
    ax.set_title(f'{col} Distribution Drift Check')
    ax.legend()

# 남는 subplot 비우기
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [0]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
fig.suptitle('Categorical Distribution Comparison', fontsize=16)

for ax, col in zip(axes, ['gender', 'grade']):
    ref_counts = df_ref[col].astype(str).value_counts(normalize=True)
    new_counts = df_recent[col].astype(str).value_counts(normalize=True)
    
    categories = list(set(ref_counts.index) | set(new_counts.index))
    categories.sort()  # 라벨 순서 정렬
    x = np.arange(len(categories))
    width = 0.35
    
    # 옆으로 나란히 막대
    ax.bar(x - width/2, [ref_counts.get(cat, 0) for cat in categories], width=width, color='skyblue', label='Reference')
    ax.bar(x + width/2, [new_counts.get(cat, 0) for cat in categories], width=width, color='orange', label='Recent')
    
    ax.set_xticks(x)
    ax.set_xticklabels(categories)
    ax.set_title(col.capitalize())
    ax.set_ylabel('Proportion')
    ax.set_xlabel(col.capitalize())
    ax.legend()
    
    # 막대 위 값 표시
    for i, cat in enumerate(categories):
        ax.text(i - width/2, ref_counts.get(cat,0)+0.01, f"{ref_counts.get(cat,0):.2f}", ha='center')
        ax.text(i + width/2, new_counts.get(cat,0)+0.01, f"{new_counts.get(cat,0):.2f}", ha='center')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


## 조건부 Job

Drift 5개 이상시 재학습

In [0]:
from mlflow.tracking import MlflowClient

# 1️⃣ Drift 체크 (예: 5개 이상 컬럼 유의한 변화)
num_drifted_cols = sum(drift_flags_numeric.values()) + sum(drift_flags_categorical.values())
if num_drifted_cols >= 5:
    # 2️⃣ 재학습 수행
    print("Drift 감지: 재학습 시작")
    drift_flag = True
    import mlflow

    MLFLOW_EXPERIMENT = "/Users/1dt003@msacademy.msai.kr/drift_monitoring"
    mlflow.set_experiment(MLFLOW_EXPERIMENT)

    with mlflow.start_run(run_name="drift_monitoring") as run:
        for col, flag in drift_flags.items():
            mlflow.log_metric(f"{col}_drift", int(flag))

        for col, flag in drift_flags_numeric.items():
            mlflow.log_metric(f"{col}_numeric_drift", int(flag))
            mlflow.log_metric(f"{col}_numeric_pvalue", p_values_numeric[col])

        for col, flag in drift_flags_categorical.items():
            mlflow.log_metric(f"{col}_categorical_drift", int(flag))
            mlflow.log_metric(f"{col}_categorical_pvalue", p_values_categorical[col])
        print("Drift metrics logged to MLflow")

else:
    drift_flag = False
    print("Drift 미충족: 재학습 생략")

dbutils.notebook.exit(str(drift_flag)) # Databricks에서 한 Notebook(Task)을 다른 Notebook(Task)에서 호출할 때, 종료값(return value)을 전달할 수 있게 해주는 함수. Job에서 이 값을 기준으로 condition조정 가능